## This notebook is an example of how we generate the input for EDCR, in the form of a .npy file
### The columns in the .npy file are as follows:
- pred: The base model's predictions	
- corr: The ground truth	
- true_positive: The TP of the base model's predictions	
- false_positive: The FP of the base model's predictions	
- ruleX: The rule results that you want to use to correct the base model's predictions
    - In our case, ruleX are generated by using various other models of different hyperparameters to perform the same binary classification task. Since we have a wide range of model results, some good, some not so good, this gives us a great variety in rules. Now that we have these new sets of predictions from various models, we use them to "correct" the base model's predictions and those "corrected" results are what's used as ruleX.
    - How these "corrections" work is similar to an ensemble classification style. We first set confidence thresholds [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]. And if correction model's prediction for a certain row is '1' (our class of interest), and if it exceeds the confidence threshold we set, we will then flip the base model's '0' prediction to a '1'. Note that we are ONLY interested in flipping '0' -> '1', and not the other way around.
    - Here's an example: 
        - Base Model LSTM predicts [0, 0, 0, 1].
        - Correction Model CNN Attention predicts [1, 0, 1, 0].
        - For the Correction Model's predictions, here are its confidence values for each one [0.9, 0.5, 0.4, 0.9].
        - Now, I've set a confidence threshold of 0.8. 
        - In the "correction" phase, the Correction Model converts the Base Model's predictions to [1, 0, 0, 1].
        - Thus, [1, 0, 0, 1] is now a ruleX!
    - Also, I'm using the term "Correction" alot here, but do not confuse it with EDCR's own "Correction" logic. This is just our way of generating rules.

In [1]:
import sys
sys.path.append('../')

from utils import rule_filter as rf

### Hyperparameter of the models

In [2]:
filters = 32
kernels = 5
units = 128

### Models used

In [3]:
CNN = f'CNN_{filters}_filters_{kernels}_kernels_predictions'
CNNA = f'CNN_Attention_{filters}_filters_{kernels}_kernels_predictions'
RNN = f'RNN_{units}_units_predictions'
LSTM = f'LSTM_{units}_layers_predictions'

### Constants

In [4]:
MODEL = LSTM
COMMODITY = 'nickel_shift_new_20'
RULE_NUM = 20
THRESHOLD = 0.35
EXCLUDE = ['CNNA']
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']
ALGO = algo[1]

model_name = 'CNN_256_filters_5_kernels_predictions'
df_path = f'../{COMMODITY}/test/predictions/test/{model_name}.csv'


## Rule Filtering Algorithms
- Since we have multiple permutations of models, confidence thresholds, and even correction algorithms, we have ALOT of rules generated. 
- EDCR's algorithm generally doesn't work super well if we just simply pass in several thousand rules, and the runtime is quite intensive as well.
- So to mitigate that, we constructed 2 rule filtering algorithms, so that we limit the number of rules we pass in, as well as control the quality of rules being learned.

### Check the *edcr_rule_filtering.ipynb* notebook for more details.

In [5]:
df = rf.npy_to_top_n_f1_bowpy(df_path, f'../{COMMODITY}/test/predictions/test_{ALGO}', RULE_NUM, None)
df

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Excluded 0 rules due to model filtering


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4,rule5,...,rule10,rule11,rule12,rule13,rule14,rule15,rule16,rule17,rule18,rule19
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.to_numpy().dump('data/test.npy')

In [7]:
import example
df = example.run_edcr()
df

TP:37, FP:63, TN:306, FN:24
ep:0.001
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.002
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.003
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.004
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.005
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.006
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.007
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.008
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.009000000000000001
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37, FP:63, TN:306, FN:24
ep:0.01
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
TP:37,

,epsilon,pre,recall,F1,NSC,PSC,NRC,PRC
0,0.001,0.37,0.606557,0.459627,0,28,0,4
1,0.002,0.37,0.606557,0.459627,0,28,0,4
2,0.003,0.37,0.606557,0.459627,0,28,0,4
3,0.004,0.37,0.606557,0.459627,0,28,0,4
4,0.005,0.37,0.606557,0.459627,0,28,0,4
...,...,...,...,...,...,...,...,...
94,0.095,0.37,0.606557,0.459627,0,28,0,4
95,0.096,0.37,0.606557,0.459627,0,28,0,4
96,0.097,0.37,0.606557,0.459627,0,28,0,4
97,0.098,0.37,0.606557,0.459627,0,28,0,4
